In [1]:
import os
import gym
import math
import argparse
from stable_baselines3 import TD3
from stable_baselines3.common import logger
from stable_baselines3.common.env_util import make_vec_env
from sklearn.model_selection import ParameterGrid

/home/t/anaconda3/envs/t/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Funkcja rozdzielająca listę komend na kilka list, aby można było poszczególne listy rozdzielić do plików
def split_list(list_to_split, n_parts):
    part_len = math.floor(len(list_to_split) / n_parts)
    remainder = len(list_to_split) % n_parts
    parts_index = []
    
    for i in range(n_parts):
        parts_index.append([i*part_len, i*part_len+part_len])
    
    if remainder != 0:
        parts_index[-1][1] = len(list_to_split)
        
    return [list_to_split[idx[0]:idx[1]] for idx in parts_index]

# W poniższych dwóch sekcjach zostały wygenerowane komendy do uruchomienia testów hiperparametrów

Poniższy kod generuje komendy do uruchomienia algorytmów TD3 i FastACER na obu środowiskach Mujoco i pybullet. Testy nie były uruchamiane sekwencyjnie ze względu na czas jaki zajmuje uruchomienie jednego testu. Komendy były uruchamiane po kilka na raz. Następnie przebieg uczenia był monitorowany przez panel tensorboard.

Komendy były rozdzielone do kilku skryptów, a następnie skrypty zostały uruchomione jedocześnie.

## TD3

In [3]:
params = {
    'env_name': ['Walker2d-v3', 'Walker2DBulletEnv-v0'],
    'learning_rate': ['0.0009', '0.001'],
    'buffer_size': ['500_000', '1_000_000'],
    'batch_size': ['80', '100'],
    'gamma': ['0.99', '0.999'],
}

param_grid = ParameterGrid(params)
param_grid = list(param_grid)
param_grid_split = split_list(param_grid, 3)
total_timesteps = 200_000

for i in range(len(param_grid_split)):
    param_grid_part = param_grid_split[i]
    file = open(f'TD3_commands_{i}.sh', 'w')

    for params in param_grid_part:
        saved_name = \
            f'--env_name {params["env_name"]} ' + \
            f'--learning_rate {params["learning_rate"]} --buffer_size {params["buffer_size"]} ' + \
            f'--batch_size {params["batch_size"]} --gamma {params["gamma"]} ' + \
            f'--total_timesteps {total_timesteps}'
        file.write(f'python TD3.py {saved_name} &&\n')

    file.write('echo Optymalizacja skończona')
    file.close()

## FastACER

In [9]:
params = {
    'env_name': ['Walker2d-v3', 'Walker2DBulletEnv-v0'],
    'gamma': ['0.99', '0.999'],
    'actor_lr': ['0.001', '0.0011'],
    'critic_lr': ['0.001', '0.0011'],
    'memory_size': ['500_000', '1_000_000'],
    'actor_layers': ['200'],
    'critic_layers': ['200']
}

param_grid = ParameterGrid(params)
param_grid = list(param_grid)
param_grid_split = split_list(param_grid, 3)
total_timesteps = 500_000

for i in range(len(param_grid_split)):
    param_grid_part = param_grid_split[i]
    file = open(f'FastACER_commands_{i}.sh', 'w')

    for params in param_grid_part:
        file.write(
            f'python acer/run.py --algo fastacer --env_name {params["env_name"]} --no_checkpoint ' + \
            f'--evaluate_time_steps_interval 50000 --num_evaluation_runs 10 ' + \
            f'--max_time_steps {total_timesteps} ' + \
            f'--gamma {params["gamma"]} --actor_lr {params["actor_lr"]} --critic_lr {params["critic_lr"]} ' + \
            f'--memory_size {params["memory_size"]} ' + \
            f'--actor_layers {params["actor_layers"]} {params["actor_layers"]} ' + \
            f'--critic_layers {params["critic_layers"]} {params["critic_layers"]} &&\n')

    file.write('echo Optymalizacja skończona')
    file.close()